# Lab 2 - Elastic Search
1. Install ElasticSearch (ES).
2. Install an ES plugin for Polish https://github.com/allegro/elasticsearch-analysis-morfologik 
3. Define an ES analyzer for Polish texts containing:
   1. standard tokenizer
   2. synonym filter with alternative forms for months, e.g. `kwiecień`, `kwi`, `IV`.
   3. lowercase filter
   4. Morfologik-based lemmatizer
   5. lowercase filter (looks strange, but Morfologi produces capitalized base forms for proper names, so we have to lowercase them once more).
4. Define another analyzer for Polish, without the synonym filter.
5. Define an ES index for storing the contents of the corpus [FiQA-PL](https://huggingface.co/datasets/clarin-knext/fiqa-pl) using both analyzers.
   Use different names for the fields analyzed with a different pipeline.

In [1]:
import os
import numpy as np
from dotenv import load_dotenv
from datasets import load_dataset
from elasticsearch import Elasticsearch, helpers

load_dotenv(override=True)

username = os.getenv('ES_USERNAME')
password = os.getenv('ES_PASSWORD')

es = Elasticsearch(
    "http://localhost:9200",
    http_auth=(username, password)
)

es.info()['version']

c:\Users\katar\miniconda3\envs\NLP\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'number': '8.15.3',
 'build_flavor': 'default',
 'build_type': 'zip',
 'build_hash': 'f97532e680b555c3a05e73a74c28afb666923018',
 'build_date': '2024-10-09T22:08:00.328917561Z',
 'build_snapshot': False,
 'lucene_version': '9.11.1',
 'minimum_wire_compatibility_version': '7.17.0',
 'minimum_index_compatibility_version': '7.0.0'}

In [3]:
index_config = {
    "settings": {
        "analysis": {
            "tokenizer": {
                "standard_tokenizer": {
                    "type": "standard"
                }
            },
            "filter": {
                "polish_synonyms": {
                    "type": "synonym",
                    "synonyms": [
                        "styczeń, sty, I",
                        "luty, lut, II",
                        "marzec, mar, III",
                        "kwiecień, kwi, IV",
                        "maj, V",
                        "czerwiec, cze, VI",
                        "lipiec, lip, VII",
                        "sierpień, sie, VIII",
                        "wrzesień, wrz, IX",
                        "październik, paź, X",
                        "listopad, lis, XI",
                        "grudzień, gru, XII"
                    ]
                },

            },
            "analyzer": {
                "polish_analyzer_with_synonyms": {
                    "type": "custom",
                    "tokenizer": "standard_tokenizer",
                    "filter": [
                        "polish_synonyms",
                        "lowercase",
                        "morfologik_stem",
                        "lowercase"
                    ]
                },
                "polish_analyzer_without_synonyms": {
                    "type": "custom",
                    "tokenizer": "standard_tokenizer",
                    "filter": [
                        "lowercase",
                        "morfologik_stem",
                        "lowercase"
                    ]
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "content_with_synonyms": {
                "type": "text",
                "analyzer": "polish_analyzer_with_synonyms"
            },
            "content_without_synonyms": {
                "type": "text",
                "analyzer": "polish_analyzer_without_synonyms"
            }
        }
    }
}


index_name = "fiqa-pl-corpus"
if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)
es.indices.create(index=index_name, body=index_config)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'fiqa-pl-corpus'}

6. Load the data to the ES index.

In [4]:
dataset = load_dataset("clarin-knext/fiqa-pl", "corpus")['corpus']

In [5]:
def generate_documents(dataset):
    for item in dataset:
        yield {
            "_index": "fiqa-pl-corpus",
            "_source": {
                "content_with_synonyms": item["text"],
                "content_without_synonyms": item["text"]
            }
        }

helpers.bulk(es, generate_documents(dataset))


(57638, [])

7. Determine the number of documents and the number of matches containing the word kwiecień (in any form) including and excluding the synonyms.

In [63]:
with_synonyms_count = es.count(index="fiqa-pl-corpus", body={
    "query": {
        "match": {
            "content_with_synonyms": "kwiecień"
        }
    }
})

without_synonyms_count = es.count(index="fiqa-pl-corpus", body={
    "query": {
        "match": {
            "content_without_synonyms": "kwiecień"
        }
    }
})

print(f"Documents containing 'kwiecień' (with synonyms): {with_synonyms_count['count']}")
print(f"Documents containing 'kwiecień' (without synonyms): {without_synonyms_count['count']}")

Documents containing 'kwiecień' (with synonyms): 306
Documents containing 'kwiecień' (without synonyms): 257


In [65]:
documents_with_kwi = es.count(index="fiqa-pl-corpus", body={
    "query": {
        "match": {
            "content_without_synonyms": "kwi"
        }
    }
})

documents_with_IV = es.count(index="fiqa-pl-corpus", body={
    "query": {
        "match": {
            "content_without_synonyms": "IV"
        }
    }
})

print(f"Documents containing 'kwi': {documents_with_kwi['count']}")
print(f"Documents containing 'IV': {documents_with_IV['count']}")

Documents containing 'kwi': 3
Documents containing 'IV': 47


In [ ]:
documents_with_kwiecien_and_kwi = es.count(index="fiqa-pl-corpus", body={
    "query": {
        "bool": {
            "must": [
                {"match": {"content_without_synonyms": "kwiecień"}},
                {"match": {"content_without_synonyms": "kwi"}}
            ]
        }
    }
})

print(f"Documents containing both 'kwiecień' and 'kwi': {documents_with_kwiecien_and_kwi['count']}")


Documents containing both 'kwiecień' and 'kwi': 1


Suma dokumantów zawierających kwiecień (bez synonimów), kwi oraz IV jest o 1 większa niż liczba dokumntów zawierających kwiecień z synonimami. Wynika to z faktu, że istnieje dokument, w którym występują dwie formy wyrazu - kwiecień i kwi.

8. Download the QA pairs for the FiQA-PL dataset.

In [14]:
queries = load_dataset("clarin-knext/fiqa-pl", "queries")['queries'].to_pandas()

qrels = load_dataset("clarin-knext/fiqa-pl-qrels")['test'].to_pandas()


9. Compute NDCG@5 for the QA dataset (the test subset) for the following setusp:
   * synonyms enabled and disabled,
   * lemmatization in the query enabled and disabled.

In [6]:
index_body = {
    "settings": {
        "analysis": {
            "filter": {
                "polish_synonym": {
                    "type": "synonym",
                    "synonyms": [
                        "styczeń, sty, I",
                        "luty, lut, II",
                        "marzec, mar, III",
                        "kwiecień, kwi, IV",
                        "maj, V",
                        "czerwiec, cze, VI",
                        "lipiec, lip, VII",
                        "sierpień, sie, VIII",
                        "wrzesień, wrz, IX",
                        "październik, paź, X",
                        "listopad, lis, XI",
                        "grudzień, gru, XII"
                    ]
                }
            },
            "analyzer": {
                "analyzer_with_synonyms_and_lemmas": {
                    "type": "custom",
                    "tokenizer": "standard",
                    "filter": [
                        "polish_synonym",
                        "lowercase",
                        "morfologik_stem",
                        "lowercase"
                    ]
                },
                "analyzer_with_synonyms_without_lemmas": {
                    "type": "custom",
                    "tokenizer": "standard",
                    "filter": [
                        "polish_synonym",
                        "lowercase"
                    ]
                },
                "analyzer_without_synonyms_with_lemmas": {
                    "type": "custom",
                    "tokenizer": "standard",
                    "filter": [
                        "lowercase",
                        "morfologik_stem",
                        "lowercase"
                    ]
                },
                "analyzer_without_synonyms_and_lemmas": {
                    "type": "custom",
                    "tokenizer": "standard",
                    "filter": [
                        "lowercase"
                    ]
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "text": {
                "type": "text",
                "analyzer": "analyzer_without_synonyms_and_lemmas"
            },
            "text_synonyms_lemmas": {
                "type": "text",
                "analyzer": "analyzer_with_synonyms_and_lemmas"
            },
            "text_synonyms": {
                "type": "text",
                "analyzer": "analyzer_with_synonyms_without_lemmas"
            },
            "text_lemmas": {
                "type": "text",
                "analyzer": "analyzer_without_synonyms_with_lemmas"
            }
        }
    }
}

index_name = "qa_index_nodcg"
if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)
es.indices.create(index="qa_index_nodcg", body=index_body)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'qa_index_nodcg'}

In [7]:
actions = dataset.map(lambda item: {
    "_op_type": "index",
    "_index": "qa_index_nodcg",
    "_source": {
        "text": item["text"], 
        "text_synonyms_lemmas": item["text"],
        "text_synonyms": item["text"], 
        "text_lemmas": item["text"],
        "id": item["_id"] 
    }
}).to_list()

helpers.bulk(es, actions)

(57638, [])

In [8]:
def execute_query(query, analyzer, field):
    response = es.search(
        index="qa_index_nodcg",
        body={
            "query": {
                "match": {
                    field: {
                        "query": query,
                        "analyzer": analyzer
                    }
                }
            },
            "size": 5
        }
    )
    return [hit["_id"] for hit in response["hits"]["hits"]]

In [9]:
import math
def count_ndcg5(answers, correct_answers):
    DCG = sum(
    1 / math.log(i + 2, 2) for i in range(5) if int(answers[i]) in correct_answers
    )
    IDCG = sum(
        1 / math.log(i + 2, 2) for i in range(min(len(correct_answers), 5))
    )

    return DCG / IDCG if IDCG > 0 else 0.0

In [10]:

analyzers = {"analyzer_with_synonyms_and_lemmas": "text_synonyms_lemmas",
             "analyzer_with_synonyms_without_lemmas": "text_synonyms",
             "analyzer_without_synonyms_with_lemmas": "text_lemmas",
             "analyzer_without_synonyms_and_lemmas": "text"}

In [11]:
def find_answers(query):
    answers = {}
    for analyzer, field in analyzers.items():
        answers[analyzer] = execute_query(query, analyzer, field)
    return answers

In [12]:
def calculate_results(answers, ids_correct):
    values = {}
    for analyzer, result in answers.items():
        values[analyzer] = count_ndcg5(result, ids_correct)
    return values

In [17]:
results = []
for id in qrels["query-id"].unique():
    question = queries[queries['_id'] == str(id)]["text"].to_list()[0]
    answers = find_answers(question)
    
    res = calculate_results(answers, qrels[qrels["query-id"] == id]["corpus-id"].to_list())
    results.append(res)

result_values = {key: [] for key in analyzers.keys()}

for res in results:
    for key in analyzers.keys():
        result_values[key].append(res[key])

for key in analyzers.keys():
    mean_value = np.mean(result_values[key])
    print(f"{key}: {mean_value}")

analyzer_with_synonyms_and_lemmas: 0.1851291130797741
analyzer_with_synonyms_without_lemmas: 0.13854570378524392
analyzer_without_synonyms_with_lemmas: 0.1851291130797741
analyzer_without_synonyms_and_lemmas: 0.13854570378524392


Uwzględnienie synonimów nie wpłynęło na wyniki, natomiast lematyzacja poprawia wyniki wyszukiwania.

### What are the strengths and weaknesses of regular expressions versus full text search regarding processing of text?

Wyrażenia regularne mogą być łatwiejsze i szybsze w użyciu w przypadku dobrze zdefiniowanych wzorców, które występują w ściśle określonej formie, jak np. adresy email, godziny. Problem pojawia się, gdy musimy rozważyć wiele form, uwzględnić odmianę wyrazów czy synonimy. Wówczas zdecydowanie lepszym rozwiązaniem będzie wyszukiwanie pełnotekstowe, które oferuje znacznie szersze możliwości, takie jak lematyzacja, obsługa synonimów oraz wyszukiwanie z tolerancją na błędy, co pozwala na bardziej elastyczne wyszukiwanie, niewymagające ręcznego wypisywania wszystkich możliwych form. Poza tym wyszukiwanie pełnotekstowe umożliwia bardziej kontekstowe przeszukiwanie dokumentów, jak np. wyszukiwanie odpowiedzi na pytania, gdzie nie szukamy dokładnego dopasowania wzrorca, a potrzebujemy uwzględnić bliskość znaczeniową.

### Can an LLM be applied in the context of searching for documents? Justify your answer, excluding the obvious observation that an LLM can be used to formulate the answer.

Tak, LLM może być wykorzystany do wyszukiwania dokumentów. Podobnie jak w przypadku Elasticsearcha, możemy stworzyć indeks dokumentów, przekształcić go na postać wektorową, która stanowi reprezentację semantyczną, a następnie przekazać taki indeks do modelu. Modele językowe są w stanie przeszukiwać ten indeks i zwracać dokumenty najistotniejsze dla naszego zapytania. Modele potrafią rozpoznawać i dopasowywać zapytania do dokumentów na poziomie semantycznym, a nie tylko leksykalnym, co oznacza, że mogą znaleźć odpowiednie dokumenty, nawet jeśli nie zawierają tych samych fraz co zapytanie. LLM są też zdolne do identyfikacji kluczowych informacji w dokumentach i tworzenia streszczeń czy klasyfikacji.  Możemy też prosić model, aby zwrócił nam dokumenty, na podstawie których wygenerował odpowiedź, dzięki czemu jesteśmy w stanie zweryfikować jej poprawność.